In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
static_df = pd.read_csv('./data_20230101_static.csv')

In [3]:
timeseries_df = pd.read_csv('./data_20221229_timeseries.csv')

In [4]:
timeseries_df.drop(['Albumin', 'Total Protein','Compliance'], axis=1, inplace=True)

In [5]:
timeseries_df.head()

,subject_id,stay_id,to_date,Platelets x1000,WBC x1000,Hgb,Total Bilirubin,PO2,PaCO2,Glucose,...,dbp_ni,mbp_ni,FiO2,Minute Ventilation,Mean Airway Pressure,Peak Airway Pressure,PEEP,Plateau,Tidal Volume,Respiratory Rate
0,10001884,37510196,2131-01-11,171.0,18.4,31.7,0.4,49.0,69.0,148.0,...,72.761905,85.761905,58.333333,9.900000,11.360000,28.400000,5.033333,23.0,379.750000,18.000000
1,10001884,37510196,2131-01-12,149.0,12.0,31.0,0.2,65.0,63.0,145.0,...,78.937500,94.781250,44.285714,9.444444,8.350000,18.166667,5.000000,NaN,369.833333,20.184211
2,10001884,37510196,2131-01-13,144.0,21.0,30.5,NaN,69.0,53.0,199.0,...,76.958333,96.166667,40.000000,10.261111,9.550000,22.333333,5.090909,17.4,418.384615,20.500000
3,10001884,37510196,2131-01-14,119.0,12.6,31.6,0.4,91.0,61.0,165.0,...,82.909091,98.727273,40.000000,9.441176,10.042857,23.428571,5.000000,20.0,410.428571,20.724138
4,10001884,37510196,2131-01-15,109.0,12.1,31.4,0.5,NaN,NaN,170.0,...,77.541667,92.750000,40.000000,9.561111,10.566667,23.500000,5.000000,NaN,341.333333,20.100000


In [6]:
timeseries_df.columns

Index(['subject_id', 'stay_id', 'to_date', 'Platelets x1000', 'WBC x1000',
       'Hgb', 'Total Bilirubin', 'PO2', 'PaCO2', 'Glucose', 'BUN', 'pH',
       'Sodium', 'Potassium', 'Magnesium', 'Calcium', 'Chloride', 'creatinine',
       'HCO3', 'Phosphate', 'Alkaline Phos.', 'AST (SGOT)', 'ALT (SGPT)',
       'PT-INR', 'SaO2', 'Heart Rate', 'sbp', 'dbp', 'mbp', 'sbp_ni', 'dbp_ni',
       'mbp_ni', 'FiO2', 'Minute Ventilation', 'Mean Airway Pressure',
       'Peak Airway Pressure', 'PEEP', 'Plateau', 'Tidal Volume',
       'Respiratory Rate'],
      dtype='object')

In [7]:
subject_ids = static_df['subject_id'].unique()

In [8]:
len(subject_ids)

5158

In [9]:
N = 7

In [10]:
X_datas = []
Y_datas = []

In [11]:
for subject_id in tqdm(subject_ids):
    q_static = static_df[static_df['subject_id'] == subject_id]
    q_timeseries = timeseries_df[timeseries_df['subject_id'] == subject_id]
    q_timeseries = q_timeseries.interpolate(method='linear',limit_direction='both', inplace=False)
    q_timeseries = q_timeseries.interpolate(method='linear',limit_direction='forward', inplace=False)
    q_timeseries = q_timeseries.interpolate(method='linear',limit_direction='backward', inplace=False)
    for col in timeseries_df.drop(columns=['subject_id', 'stay_id', 'to_date']).columns:
        q_timeseries[col] = q_timeseries[col].apply(lambda x : x if pd.notna(x) else timeseries_df[col].mean())
    # q_timeseries = q_timeseries.fillna(0)
    if len(q_timeseries) >= 7:
        q_timeseries = q_timeseries.drop(columns=['subject_id', 'stay_id', 'to_date'])
        if len(q_static) != 0 and len(q_timeseries) != 0:
            X_datas.append(q_timeseries[-N:].to_numpy())
            Y_datas.append(q_static['dod'].isna().astype(np.int64).iloc[0])


100%|██████████| 5158/5158 [01:47<00:00, 48.16it/s]


In [12]:
X_datas = np.array(X_datas)
Y_datas = np.array(Y_datas)

In [13]:
X_datas.shape

(5152, 7, 37)

In [14]:
Y_datas.shape

(5152,)

In [15]:
assert Y_datas.shape[0] == X_datas.shape[0]

In [16]:
np.save('timeseries_Datas.npy', X_datas)
np.save('dod_Datas.npy', Y_datas)